# Prepration

Loading env variables and vectorDB

In [ ]:
from dotenv import load_dotenv
import os
import pickle

# Laden Sie die Umgebungsvariablen aus der .env-Datei
load_dotenv()
API_KEY = os.environ.get("API_KEY")

In [ ]:
with open("vectorstore.pkl", "rb") as f:
    vectorstore = pickle.load(f)

# Agents

Agenten verwenden ein LLM, um zu bestimmen, welche Aktionen in welcher Reihenfolge durchgeführt werden sollen. Eine Aktion kann entweder die Verwendung eines Werkzeugs und die Beobachtung seiner Ausgabe oder die Rückgabe an den Benutzer sein. Für die Verwendung eines Agent ist es wichtig neben dem Konzept eines LLM, ein neues Konzept und das eines "Tools" zu verstehen.

## Tools

Werkzeuge sind Funktionen, die Agenten nutzen können, um mit der Welt zu interagieren. Diese Werkzeuge können allgemeine Dienstprogramme (z. B. Suche), andere Ketten oder sogar andere Agenten sein.

In [ ]:
from langchain.agents import load_tools
from langchain.llms import OpenAI

llm=OpenAI(model_name="text-davinci-003", temperature=0.7, openai_api_key=API_KEY)

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

In [ ]:
from langchain.agents import Tool
tool_list = [
    Tool(
        name = "Math Tool",
        func=tools[0].run,
        description="Tool to calculate, nothing else"
    )
]

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(tool_list, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)
agent.run("Wie geht es dir?")

In [ ]:
agent.run("Was ist 100 geteilt durch 25?")

# Custom Tools

Man kann auch eigene Tools erstellen indem man eine Klasse erstellt, die von BaseTool erbt.

In [ ]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun

class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer questions about current events"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = vectorstore.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)
    
    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [ ]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [ ]:
agent.run("Was ist ein Huninchen?")